# 1.1 Clean up and raw generate a csv file.

In [1]:
import re
import os
import pandas as pd


greeks_list = ["\alpha","\beta","\gamma","\Gamma","\delta","\Delta","\epsilon","\varepsilon",
             "\zeta","\eta","\theta","\vartheta","\Theta","\iota","\kappa","\lambda","\Lambda",
             "\mu","\nu","\pi","\Pi","\rho","\varrho","\sigma","\Sigma","\tau","\phi","\varphi","\Phi",
             "\chi","\psi","\Psi","\omega","\Omega"]

# input a sample  tex
def generateDF(file, cat = "Unknown", name = 'Unknown', path = './CSVs/'):
    # clean up
    with open(file, 'r') as myfile:
        al = " ".join(myfile.readlines())

        brackets = re.compile('\[((.|\n)*?)\]')
        newal = brackets.sub('', al)

        comments = re.compile('\%.*')
        newal = comments.sub('\n', newal)
        
        whitespace = re.compile('(?!\w|\})\s+(?=\{)')
        newal = whitespace.sub('', newal)
        
        redundantNewlines = re.compile('\s\s+')
        newal = redundantNewlines.sub('\n', newal)
        
        firstCurly = re.compile('\{\n')
        newal = firstCurly.sub('\{', newal)
        
        lastCurly = re.compile('\n\}')
        newal = lastCurly.sub('}', newal)
        
        forceNewline = re.compile('(\\\\\n)')
        newal = forceNewline.sub('\\\\ ', newal)
        
        mathmodeRm = re.compile('\$.*?\$')
        newal = mathmodeRm.sub(' ', newal)
        
#         print(newal)
#         print(repr(newal))

    # write in clean up
    with open('cleanup.tex', 'w') as myfilecl:
        myfilecl.write(newal)
        
    with open('cleanup.tex', 'r') as myfile:
        i = 0
        matches = pd.DataFrame(columns = ['Line num','Command', 'Notes'])

        for idx, line in enumerate(myfile):
            i += 1
            env = False
            newcmd = False
            nested = False
            match = re.findall(r'\\\w+', line)

            if '\\end' in match:
                match = []

            if '\\begin' in match:
                env = True
                match = [re.findall('(?!\\begin{)\w+', line)[1]]
                note = ['Environment']
                
            if '\\newcommand' in match:
                newcmd = True
                note = re.findall('\{(.*)\}', line)[0]
                p = re.compile('\}\{')
                note = p.sub(' -> ', note)
                match = ['\\newcommand']
                
            for g in greeks_list:
                match[:] = [x for x in match if x != g]
                
#             print(match)

            for idx, j in enumerate(match):
                if '\\usepackage' in match:
                    note = re.findall('\{(.*?)\}', line)

                elif '\\documentclass' in match:
                    note = re.findall('\{(.*?)\}', line)

                elif '\\newcommand' in match:
                    pass
                
                elif '\\cite' in match:
                    note = re.findall('\{(.*)\}', line)
                    
                elif '\\ref' in match:
                    note = re.findall('\{(.*)\}', line)
                    
                elif env is True:
                    env = False
                    pass
                
                elif len(re.findall('\{(.*?)\}', line)) == 1:
                    note = re.findall('\{(.*?)\}', line)
                    
                else:
                    note = ['N/A']
                
                notelen = len((re.findall(r'\\', str(note))))
                
                if notelen == 0 or newcmd is True or j == '\\author' or j == '\\title':
                    new_row = pd.DataFrame({'Line num': i, 'Command': [j], 'Notes': note})                   
                    matches = matches.append(new_row, ignore_index=True)
                else:
                    new_row = pd.DataFrame({'Line num': i, 'Command': [j], 'Notes': 'Nested'})                   
                    matches = matches.append(new_row, ignore_index=True)
                    
                # this means that there exists nested commands
                
                if notelen != 0:
                    if newcmd is True:
                        newcmd = False
                        pass
                    else:
                        subnotes = re.findall(r'(\\\w+\{.*?\})', str(note[0]))
#                         print(str(note[0]), subnotes)
                        for subnote in subnotes:
#                             print(re.findall(r'\{(.*)\}', subnote)[0])
                            new_row = pd.DataFrame({'Line num': i,
                                                    'Command': re.findall(r'\\\w+', subnote), 
                                                    'Notes': re.findall(r'\{(.*)\}', subnote)[0]})
                            matches = matches.append(new_row, ignore_index=True)
                        
                
                
#     pd.set_option('display.max_columns', None)  # or 1000
#     pd.set_option('display.max_rows', None)  # or 1000
#     pd.set_option('display.max_colwidth', -1)  # or 199
#     print(matches)
                
    matches.to_csv(path + cat + '-' + name + '.csv')

## Some test

In [2]:
# test
generateDF('sample.tex', name = 'sample', path = './')

# 1.2 Going through CSVs we have harvested in r

In [4]:
unzipped_path = './Download/unzipped/'

categories = ["math.AC-CommutativeAlgebra", "math.AG-AlgebraicGeometry", "math.AP-AnalysisofPDEs", 
    "math.AT-AlgebraicTopology", "math.CA-ClassicalAnalysisandODEs", "math.CO-Combinatorics", "math.CT-CategoryTheory", 
    "math.CV-ComplexVariables", "math.DG-DifferentialGeometry", "math.DS-DynamicalSystems", "math.FA-FunctionalAnalysis", 
    "math.GM-GeneralMathematics", "math.GN-GeneralTopology", "math.GR-GroupTheory", "math.GT-GeometricTopology", 
    "math.HO-HistoryandOverview", "math.IT-InformationTheory", "math.KT-K-TheoryandHomology", "math.LO-Logic", 
    "math.MG-MetricGeometry", "math.MP-MathematicalPhysics", "math.NA-NumericalAnalysis", "math.NT-NumberTheory", 
    "math.OA-OperatorAlgebras", "math.OC-OptimizationandControl", "math.PR-Probability", "math.QA-QuantumAlgebra", 
    "math.RA-RingsandAlgebras", "math.RT-RepresentationTheory", "math.SG-SymplecticGeometry", "math.SP-SpectralTheory", "math.ST-StatisticsTheory"]

for cat in categories:
    cat_path = unzipped_path + cat
    file_paths = [x[0] for x in os.walk(cat_path)][1:]

    for file_path in file_paths:
        for file in os.listdir(file_path):
            if file.endswith(".tex"):
                with open(file_path + '/' + file) as f:
                    try:
                        if '\\documentclass' in f.read():
                            generateDF(file_path + '/' + file, cat[5:7], file[:-4])
                    except:
                        pass

# 2.1 Generate summary CSV for each CSV

In [5]:
import csv
import pandas as pd

def generateSummary(inp):
    sum_row_list = [[
        'title',
        'author',
        'categories',
        'total_commands',
        'num_package',
        'num_newcommand',
        'num_newtheorem',
        'num_cite',
        'num_ref',
        'num_label',
        'num_section',
        'file_name'
        ]]

    # #############starting loop through files

    data = pd.read_csv(inp, usecols=['Command', 'Notes'])

    # find author&title

    author = ''
    title = ''
    for i in range(0, len(data)):

        # record author & title

        if data['Command'][i] == '\\author':
            author += data['Notes'][i]

        if data['Command'][i] == '\\title':
            title = data['Notes'][i]

        # drop rows with empty Notes

        if data['Notes'][i] == 'N/A' or data['Notes'][i] == '':
            data.drop(index=i, inplace=True, ignore_index=True)

    total_com = len(data)

    # count commands

    keys_list = [
        'num_package',
        'num_newcommand',
        'num_newtheorem',
        'num_cite',
        'num_ref',
        'num_label',
        'num_section',
        ]
    counting = {}
    for i in keys_list:
        counting[i] = 0

    data_List = data['Command']
    for i in data_List:
        if i == '\\usepackage':
            counting['num_package'] += 1
        if i == '\\newcommand':
            counting['num_newcommand'] += 1
        if i == '\\newtheorem':
            counting['num_newtheorem'] += 1
        if i == '\\cite':
            counting['num_cite'] += 1
        if i == '\\ref':
            counting['num_ref'] += 1
        if i == '\\label':
            counting['num_label'] += 1
        if i == '\\section':
            counting['num_section'] += 1

    sum_row_list.append([
        title,
        author,
        inp[7:9],
        total_com,
        counting['num_package'],
        counting['num_newcommand'],
        counting['num_newtheorem'],
        counting['num_cite'],
        counting['num_ref'],
        counting['num_label'],
        counting['num_section'],
        inp
        ])
    
    return sum_row_list

# 2.2 Going through each file

In [6]:
import os
CSVs_path = './CSVs/'

with open('./Summary.csv', 'w', newline='') as of:
    i = 0
    for filename in os.listdir(CSVs_path):
        if filename.endswith(".csv"):
            try:
                samp = generateSummary(CSVs_path + filename)
                if i == 0:
                    writer = csv.writer(of)
                    writer.writerows(samp)
                    i += 1
                else:
                    writer = csv.writer(of)
                    writer.writerow(samp[1])
            except:
                pass

In [26]:
generateSummary('SampleCsv.csv')

[['title',
  'author',
  'total_commands',
  'num_package',
  'num_newcommand',
  'num_newtheorem',
  'num_cite',
  'num_ref',
  'num_label',
  'num_section'],
 ['On uniqueness of additive actions on complete toric varieties',
  ',Sergey Dzhunusov',
  1256,
  17,
  2,
  11,
  36,
  33,
  6,
  7]]